# Daily Historical Updates Data Ingestion

## Setting up the glue session

In [0]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas


  Obtaining dependency information for openmeteo-requests from https://files.pythonhosted.org/packages/13/31/96209383687cf35055eb628e3a9207a07ac2d5faf6e70076f459435a989e/openmeteo_requests-1.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for openmeteo-sdk>=1.4.0 from https://files.pythonhosted.org/packages/18/9a/f33c4eb783d505d0099c039bbac30da09266027d9e3e0b5de76ef796749d/openmeteo_sdk-1.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.0.0 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for requests-cache from https://files.pythonhosted.org/packages/4e/2e/8f4051119f460cfc786aa91f212165bb6e643283b533db572d7b33952bd2/requests_cache-1.2.1-py3-none-any.whl.metadata
  O

## Importing Libraries

In [0]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timedelta
from pyspark.sql.functions import col, max as spark_max
from pyspark.sql.functions import col, when
from pyspark.sql import functions as F


## Setting up parameters and requesting data using the weather API

In [0]:
# Define S3 bucket details and file path
bucket_name = "de-upskilling-weather"
folder = "LandingZone"
file_name = "daily_historical_updates.parquet"
org_file_name = "daily_historical.parquet"
s3_path = f"/mnt/{bucket_name}/{folder}/Updates/{file_name}"

df_update = spark.read.parquet(f"/mnt/{bucket_name}/Gold/log.parquet")

# Convert 'datetime' column to timestamp type
df_update = df_update.withColumn("latest_data", F.to_timestamp("latest_data"))

# Specify the name you are filtering for
specific_name = "daily_historical.parquet"

# Filter by name and select the row with the maximum datetime
last_updated = df_update.filter(F.col("file_name") == org_file_name).orderBy(F.col("latest_data").desc()).limit(1).select('latest_data').collect()[0][0].date()

# Convert last_updated to a datetime object
#last_updated = datetime.strptime(last_updated, "%Y-%m-%d")
# Set start_date to one day after the last date
start_date = (last_updated + timedelta(days=1)).strftime("%Y-%m-%d")

end_date = (datetime.today() - timedelta(days=2)).strftime("%Y-%m-%d")
    
print("Ingestion will start from:", start_date)

print("End date is set to:", end_date)

Ingestion will start from: 2024-12-16
End date is set to: 2024-12-15


In [0]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

latitudes = [38.855584, 40.724039, 25.695215, 34.005712, 32.724548, 47.594472, 39.676938, 39.952583, 39.791, 35.481918, 36.174465, 41.881832, 51.509865, 52.520008, 48.137154, 48.864716, 47.373878, 51.260197, 52.377956, 53.35014, 40.416775, 48.210033, 46.056946, 50.073658, 55.676098]
longitudes = [-77.036975, -73.994982, -80.168933, -118.175596, -96.76923, -122.348286, -104.977053, -75.165222, -86.148003, -97.508469, -86.76796, -87.623177, -0.118092, 13.404954, 11.576124, 2.349014, 8.545094, 4.402771, 4.89707, -6.266155, -3.70379, 16.363449, 14.505751, 14.41854, 12.568337]
params = {
	"latitude": latitudes,
	"longitude": longitudes,
	"start_date": start_date,
	"end_date": end_date,
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"]
}
responses = openmeteo.weather_api(url, params=params)


## Processing the responce
* Iterating through the list received
* Extracting information
* storing the information in a spark df

In [0]:
# Process first location. Add a for-loop for multiple locations or weather models

for i, response in enumerate(responses):

    #response = responses[0]
    #print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    #print(f"Elevation {response.Elevation()} m asl")
    #print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    #print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(5).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(7).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(8).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(9).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(10).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(11).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}
    
    daily_data["latitude"] = latitudes[i]
    daily_data["longitude"] = longitudes[i]
    
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

    daily_dataframe = pd.DataFrame(data = daily_data)
    
    if i == 0:
        spark_df = spark.createDataFrame(daily_dataframe)
    else:
        # Convert `daily_dataframe` to a Spark DataFrame
        daily_spark_df = spark.createDataFrame(daily_dataframe)

        # Step 2: Concatenate the two Spark DataFrames
        spark_df = spark_df.unionByName(daily_spark_df)
    
    #print(daily_dataframe)

In [0]:
spark_df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- temperature_2m_max: double (nullable = true)
 |-- temperature_2m_min: double (nullable = true)
 |-- temperature_2m_mean: double (nullable = true)
 |-- apparent_temperature_max: double (nullable = true)
 |-- apparent_temperature_min: double (nullable = true)
 |-- apparent_temperature_mean: double (nullable = true)
 |-- precipitation_sum: double (nullable = true)
 |-- rain_sum: double (nullable = true)
 |-- snowfall_sum: double (nullable = true)
 |-- wind_speed_10m_max: double (nullable = true)
 |-- wind_gusts_10m_max: double (nullable = true)
 |-- wind_direction_10m_dominant: double (nullable = true)



In [0]:
spark_df.count()

250

## Writing the df into the S3 bucket LandingZone Folder and updating the last_updated dim table

In [0]:
# Write the DataFrame to an S3 bucket in Parquet format
spark_df.write.mode("overwrite").parquet(s3_path)


In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from datetime import datetime

# Get current date and time
now = datetime.now()

# Format date and time as a string
formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S")

# Sample data
data = [
    (org_file_name, end_date, formatted_date_time)
]

# Define schema
schema = StructType([
    StructField("file_name", StringType(), True),
    StructField("latest_data", StringType(), True),
    StructField("date_updated", StringType(), True)
])

# Create DataFrame
df_log = spark.createDataFrame(data, schema)

# Show DataFrame
df_log.show()

+--------------------+-----------+-------------------+
|           file_name|latest_data|       date_updated|
+--------------------+-----------+-------------------+
|daily_historical....| 2024-12-09|2024-12-11 15:28:18|
+--------------------+-----------+-------------------+



In [0]:
df_log.write.mode("append").parquet(f"/mnt/{bucket_name}/Gold/log.parquet")